In [2]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\nikgu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [5]:
url = "https://dining.ncsu.edu/location/fountain"

page = requests.get(url)

page

<Response [200]>

In [37]:
soup = bs(page.text, 'html.parser')

items = soup.find_all(class_ = "dining-menu-item-modal")
item_id = []

for item in items:
    item_id.append(item["rel"])

base_url = "https://dining.ncsu.edu/wp-admin/admin-ajax.php?action=ncdining_ajax_get_item_nutrition&item_id="
new_hrefs = []

for id in item_id:
    href = base_url + id[0]
    new_hrefs.append(href)

In [16]:
for refferal in new_hrefs:
    url = refferal

bs4.element.Tag

In [52]:
url = new_hrefs[0]
page = requests.get(url)

soup2 = bs(page.text, 'html.parser')

specific_item = soup2.find_all("div")
specific_item 

[<div class="dining-menu-item-modal-title"><div><h3>Beyond Meat Burger Patty</h3></div><div class="clearfix"></div></div>,
 <div><h3>Beyond Meat Burger Patty</h3></div>,
 <div class="clearfix"></div>,
 <div class="row"><div class="col-sm-6"><div class="menu-dining-menu-modal-nutrition"><h4>Nutrition Facts</h4><div class="menu-nutrition-row"><strong>Serving Size:</strong> <span class="menu-nutrition-row-value">SERVING (1 burger)</span></div><div class="menu-nutrition-row"><strong>Calories</strong> <span class="menu-nutrition-row-value">308.2</span></div><div class="menu-nutrition-row"><strong>Protein</strong> <span class="menu-nutrition-row-value">20</span></div><div class="menu-nutrition-row"><strong>Total Carbohydrate</strong> <span class="menu-nutrition-row-value">6</span></div><div class="menu-nutrition-row"><strong>Total Fat</strong> <span class="menu-nutrition-row-value">24</span></div><div class="menu-nutrition-row"><strong>Saturated Fat</strong> <span class="menu-nutrition-row-v

In [166]:
class Food:
    def __init__(self, name, location, meal_session, meal_date, serving_size, calories, protein, carbohydrates, 
                 total_fat, saturated_fat, trans_fat, fiber, sodium, diet_violation, allergen_violation,
                 nutrient_violation, special_violation, ingredients):
        self.name = name
        self.location = location
        self.meal_session = meal_session
        self.meal_date = meal_date
        self.serving_size = serving_size
        self.calories = calories
        self.protein = protein
        self.carbohydrates = carbohydrates
        self.total_fat = total_fat
        self.saturated_fat = saturated_fat
        self.trans_fat = trans_fat
        self.fiber = fiber
        self.sodium = sodium
        self.diet_violation = diet_violation
        self.allergen_violation = allergen_violation
        self.nutrient_violation = nutrient_violation
        self.special_violation = special_violation
        self.ingredients = ingredients

In [54]:
its = specific_item[0]
# name = its.find_all("h3")[0].get_text()
# nutrition = its.find_all("div")
its

<div class="dining-menu-item-modal-title"><div><h3>Beyond Meat Burger Patty</h3></div><div class="clearfix"></div></div>

In [158]:
date = soup.find(id = "food-menu-date").find(selected = "")["value"]
date = date[5:7] + "/" + date[8:] + "/" + date[0:4]

location = soup.find("h1").get_text()

'Fountain'

In [175]:
name = soup2.find("h3").get_text()
#location
#date

serving_size = ""
calories = ""
protein = ""
carbohydrates = ""
total_fat = ""
saturated_fat = ""
trans_fat = ""
fiber = ""
sodium = ""

nutrition_info = soup2.find_all(class_ = "menu-nutrition-row")

for nutri in nutrition_info:
    nutrition_tag = nutri.find("strong").get_text()

    if (nutrition_tag.find("Serving") != -1):
        serving_size = nutri.find("span").get_text()
        # print("Serving Size: " + serving_size)
    elif (nutrition_tag.find("Calories") != -1):
        calories = nutri.find("span").get_text()
        # print("Calories: " + calories)
    elif (nutrition_tag.find("Protein") != -1):
        protein = nutri.find("span").get_text()
        # print("Protein: " + protein)
    elif (nutrition_tag.find("Carbohydrate") != -1):
        carbohydrates = nutri.find("span").get_text()
        # print("Carbs: " + carbohydrates)
    elif (nutrition_tag.find("Total Fat") != -1):
        total_fat = nutri.find("span").get_text()
        # print("Total Fat: " + total_fat)
    elif (nutrition_tag.find("Saturated Fat") != -1):
        saturated_fat = nutri.find("span").get_text()
        # print("Saturated Fat: " + saturated_fat)
    elif (nutrition_tag.find("Trans Fat") != -1):
        trans_fat = nutri.find("span").get_text()
        # print("Trans Fat: " + trans_fat)
    elif (nutrition_tag.find("Fiber") != -1):
        fiber = nutri.find("span").get_text()
        # print("Fiber: " + fiber)
    elif (nutrition_tag.find("Sodium") != -1):
        sodium = nutri.find("span").get_text()
        # print("Sodium: " + sodium)

ingredients = soup2.find(class_ = "menu-dining-menu-modal-ingredients")
combined_ingredients = ingredients.get_text()

combined_ingredients = combined_ingredients.replace("Ingredients: ", "")


# combined_ingredients = combined_ingredients.replace("(", ", ")
# combined_ingredients = combined_ingredients.replace(")", ",")
import re
# print(re.findall(r"\(FOR.*?\)", combined_ingredients))
# print(re.findall(r"\(TO.*?\)", combined_ingredients))

to_for_substring_dict = {}

for for_substrings in re.findall(r"\(FOR.*?\)", combined_ingredients):
    to_for_substring_dict.update({for_substrings : 1})

for to_substrings in re.findall(r"\(TO.*?\)", combined_ingredients):
    to_for_substring_dict.update({to_substrings : 1})

# print(combined_ingredients)

for sub in to_for_substring_dict:
    combined_ingredients = combined_ingredients.replace(sub, "")

# print(combined_ingredients)
combined_ingredients = combined_ingredients.replace("(", ",")
combined_ingredients = combined_ingredients.replace(")", ",")
combined_ingredients = combined_ingredients.replace(",,", ",")
combined_ingredients = combined_ingredients.replace(".", "")

combined_ingredient_list = combined_ingredients.split(",")

for index in range(len(combined_ingredient_list)):
    combined_ingredient_list[index] = combined_ingredient_list[index].title().strip()

combined_ingredient_list.remove("")
combined_ingredient_list

combined_restrictions = soup2.find(class_ = "dining-menu-item-allergens").find_all("span")

cleaned_restrictions = []

for restriction in combined_restrictions:
    cleaned_restrictions.append(restriction.find("img")["alt"])

nutrient_base = ["High Carb", "Low Carb", "High Fiber", "Low Fiber", "High Protein", "Low Protein", "High Fat", "Low Fat",
                 "Low Saturated Fat", "No Trans Fat", "Low Sodium"]

diet_restriction = []
allergen_restriction = []
nutrient_restriction = []
special_restriction = []

if ("Vegan" not in cleaned_restrictions):
    diet_restriction.append("Vegan")

if ("Vegetarian" not in cleaned_restrictions):
    diet_restriction.append("Vegetarian (Ovo Lacto)")
    diet_restriction.append("Lacto Vegetarian")
    diet_restriction.append("Ovo Vegetarian")
elif ("Vegetarian" in cleaned_restrictions and "Egg Allergen" in cleaned_restrictions):
    diet_restriction.append("Vegetarian (Ovo Lacto)")
    diet_restriction.append("Ovo Vegetarian")
elif ("Vegetarian" in cleaned_restrictions and "Dairy Allergen" in cleaned_restrictions):
    diet_restriction.append("Vegetarian (Ovo Lacto)")
    diet_restriction.append("Lacto Vegetarian")

if ("Vegetarian" not in cleaned_restrictions and "Seafood Allergen" not in cleaned_restrictions):
    diet_restriction.append("Pescatarian")

if (carbohydrates != "" and float(carbohydrates) >= 50):
    diet_restriction.append("Keto/Atkins")

if ("Halal" not in cleaned_restrictions):
    diet_restriction.append("Halal")

if ("Wolf Approved" not in cleaned_restrictions):
    diet_restriction.append("Wolf Approved")

if ("Gluten Allergen" in cleaned_restrictions):
    diet_restriction.append("Gluten-Free")
    allergen_restriction.append("Wheat")

if ("Dairy Allergen" in cleaned_restrictions):
    allergen_restriction.append("Milk")

if ("Egg Allergen" in cleaned_restrictions):
    allergen_restriction.append("Egg")

if ("Seafood Allergen" in cleaned_restrictions):
    allergen_restriction.append("Fish")
    allergen_restriction.append("Shellfish")

if ("Nut Allergen" in cleaned_restrictions):
    allergen_restriction.append("Tree Nuts")
    allergen_restriction.append("Peanuts")

if ("Pork Allergen" in cleaned_restrictions):
    allergen_restriction.append("Pork")

if ("Soy Allergen" in cleaned_restrictions):
    allergen_restriction.append("Soy")

if ("Sesame Allergen" in cleaned_restrictions):
    allergen_restriction.append("Sesame")

if (carbohydrates != "" and float(carbohydrates) < (0.2 * 275)):
    nutrient_restriction.append("High Carb")

# Need clarification on Diabetic diet

meal_sesh = "Lunch"

food_item = Food(name, location, meal_sesh, date, serving_size, calories, protein, carbohydrates, total_fat, saturated_fat,
                 trans_fat, fiber, sodium, diet_restriction, allergen_restriction, nutrient_restriction, special_restriction, combined_ingredient_list)

In [192]:
date = soup.find(id = "food-menu-date").find(selected = "")["value"]
date = date[5:7] + "/" + date[8:] + "/" + date[0:4]

location = soup.find("h1").get_text()

food_list = []

count = 0

for refferal in new_hrefs:
    url = refferal
    page = requests.get(url)

    soup2 = bs(page.text, 'html.parser')

    specific_item = soup2.find_all("div")
    specific_item 

    name = soup2.find("h3").get_text()
    #location
    #date

    serving_size = ""
    calories = ""
    protein = ""
    carbohydrates = ""
    total_fat = ""
    saturated_fat = ""
    trans_fat = ""
    fiber = ""
    sodium = ""

    nutrition_info = soup2.find_all(class_ = "menu-nutrition-row")

    for nutri in nutrition_info:
        nutrition_tag = nutri.find("strong").get_text()

        if (nutrition_tag.find("Serving") != -1):
            serving_size = nutri.find("span").get_text()
            # print("Serving Size: " + serving_size)
        elif (nutrition_tag.find("Calories") != -1):
            calories = nutri.find("span").get_text()
            # print("Calories: " + calories)
        elif (nutrition_tag.find("Protein") != -1):
            protein = nutri.find("span").get_text()
            # print("Protein: " + protein)
        elif (nutrition_tag.find("Carbohydrate") != -1):
            carbohydrates = nutri.find("span").get_text()
            # print("Carbs: " + carbohydrates)
        elif (nutrition_tag.find("Total Fat") != -1):
            total_fat = nutri.find("span").get_text()
            # print("Total Fat: " + total_fat)
        elif (nutrition_tag.find("Saturated Fat") != -1):
            saturated_fat = nutri.find("span").get_text()
            # print("Saturated Fat: " + saturated_fat)
        elif (nutrition_tag.find("Trans Fat") != -1):
            trans_fat = nutri.find("span").get_text()
            # print("Trans Fat: " + trans_fat)
        elif (nutrition_tag.find("Fiber") != -1):
            fiber = nutri.find("span").get_text()
            # print("Fiber: " + fiber)
        elif (nutrition_tag.find("Sodium") != -1):
            sodium = nutri.find("span").get_text()
            # print("Sodium: " + sodium)

    ingredients = soup2.find(class_ = "menu-dining-menu-modal-ingredients")
    combined_ingredients = ingredients.get_text()

    combined_ingredients = combined_ingredients.replace("Ingredients: ", "")


    # combined_ingredients = combined_ingredients.replace("(", ", ")
    # combined_ingredients = combined_ingredients.replace(")", ",")
    import re
    # print(re.findall(r"\(FOR.*?\)", combined_ingredients))
    # print(re.findall(r"\(TO.*?\)", combined_ingredients))

    to_for_substring_dict = {}

    for for_substrings in re.findall(r"\(FOR.*?\)", combined_ingredients):
        to_for_substring_dict.update({for_substrings : 1})

    for to_substrings in re.findall(r"\(TO.*?\)", combined_ingredients):
        to_for_substring_dict.update({to_substrings : 1})

    # print(combined_ingredients)

    for sub in to_for_substring_dict:
        combined_ingredients = combined_ingredients.replace(sub, "")

    # print(combined_ingredients)
    combined_ingredients = combined_ingredients.replace("(", ",")
    combined_ingredients = combined_ingredients.replace(")", ",")
    combined_ingredients = combined_ingredients.replace(",,", ",")
    combined_ingredients = combined_ingredients.replace(".", "")

    combined_ingredient_list = combined_ingredients.split(",")

    for index in range(len(combined_ingredient_list)):
        combined_ingredient_list[index] = combined_ingredient_list[index].title().strip()

    # combined_ingredient_list.remove("")
    try:
        combined_ingredient_list.remove("")
    except:
        #empty
        count = count + 1

    # combined_ingredient_list

    combined_restrictions = soup2.find(class_ = "dining-menu-item-allergens").find_all("span")

    cleaned_restrictions = []

    for restriction in combined_restrictions:
        cleaned_restrictions.append(restriction.find("img")["alt"])

    nutrient_base = ["High Carb", "Low Carb", "High Fiber", "Low Fiber", "High Protein", "Low Protein", "High Fat", "Low Fat",
                    "Low Saturated Fat", "No Trans Fat", "Low Sodium"]

    diet_restriction = []
    allergen_restriction = []
    nutrient_restriction = []
    special_restriction = []

    if ("Vegan" in cleaned_restrictions):
        diet_restriction.append("vegan")

    if ("Vegetarian" in cleaned_restrictions):
        diet_restriction.append("vegetarian")
        if ("Egg Allergen" not in cleaned_restrictions):
            diet_restriction.append("lactovegetarian")
        else:
            if ("Dairy Allergen" not in cleaned_restrictions):
                diet_restriction.append("ovovegetarian")

    # if ("Vegetarian" not in cleaned_restrictions):
    #     diet_restriction.append("Vegetarian (Ovo Lacto)")
    #     diet_restriction.append("Lacto Vegetarian")
    #     diet_restriction.append("Ovo Vegetarian")
    # elif ("Vegetarian" in cleaned_restrictions and "Egg Allergen" in cleaned_restrictions):
    #     diet_restriction.append("Lacto Vegetarian")
    # elif ("Vegetarian" in cleaned_restrictions and "Dairy Allergen" in cleaned_restrictions):
    #     diet_restriction.append("Ovo Vegetarian")

    if ("Vegetarian" in cleaned_restrictions or "Seafood Allergen" in cleaned_restrictions):
        diet_restriction.append("Pescatarian")

    if (carbohydrates != "" and float(carbohydrates) < 50):
        diet_restriction.append("Keto")

    if ("Halal" in cleaned_restrictions):
        diet_restriction.append("Halal")

    if ("Wolf Approved" in cleaned_restrictions):
        diet_restriction.append("Wolf Approved")

    if ("Gluten Allergen" not in cleaned_restrictions):
        diet_restriction.append("Gluten-Free")
    
    if ("Gluten Allergen" in cleaned_restrictions):
        allergen_restriction.append("Wheat")

    if ("Dairy Allergen" in cleaned_restrictions):
        allergen_restriction.append("Milk")

    if ("Egg Allergen" in cleaned_restrictions):
        allergen_restriction.append("Egg")

    if ("Seafood Allergen" in cleaned_restrictions):
        allergen_restriction.append("Fish")
        allergen_restriction.append("Shellfish")

    if ("Nut Allergen" in cleaned_restrictions):
        allergen_restriction.append("Tree Nuts")
        allergen_restriction.append("Peanuts")

    if ("Pork Allergen" in cleaned_restrictions):
        allergen_restriction.append("Pork")

    if ("Soy Allergen" in cleaned_restrictions):
        allergen_restriction.append("Soy")

    if ("Sesame Allergen" in cleaned_restrictions):
        allergen_restriction.append("Sesame")

    if (carbohydrates != "" and float(carbohydrates) >= 15):
        nutrient_restriction.append("High Carb")

    # Need clarification on Diabetic diet

    meal_sesh = "Lunch"

    food_item = Food(name, location, meal_sesh, date, serving_size, calories, protein, carbohydrates, total_fat, saturated_fat,
                    trans_fat, fiber, sodium, diet_restriction, allergen_restriction, nutrient_restriction, special_restriction, combined_ingredient_list)
    
    food_list.append(food_item)

In [203]:
import json

# limit_test = food_list[0:3]

with open('test.json', 'w') as f:
    f.write("{")
    for food in food_list:
        f.write("\""+ food.name + "\"" + " :")
        json.dump(food.__dict__, f)
        if (food != limit_test[len(limit_test) - 1]):
            f.write(",")
    f.write("}")

In [187]:
food_list[0]

In [188]:
for adfsf in food_list:
    print(adfsf.name)
    print(adfsf.carbohydrates)
    print(adfsf.nutrient_violation)
    print()

Beyond Meat Burger Patty
6
['High Carb']

Cheeseburger Patty
0.5
['High Carb']

French Fries
43.1
[]

Hamburger Bun
27.7
[]

Hamburger Patty
0
['High Carb']

Hot Dog
1
['High Carb']

Hot Dog Bun
17.4
[]

Breaded Popcorn Shrimp
15.3
[]

Fresh Broccoli
5.4
['High Carb']

Garden Blend Rice
29.8
[]

Grilled Ranch Style Chicken
0
['High Carb']

Hawaiian Sweet Roll
16
[]

Lemon Cous Cous
22.8
[]

Yellow Squash with Onions
4.6
['High Carb']

Cheddar Cheese
0
['High Carb']

Ham Deli Meat
4.3
['High Carb']

Homemade Chicken Salad
5.5
['High Carb']

Kettle Chips
18
[]

Pimento Cheese 
8.5
['High Carb']

Roast Beef Deli Meat
2.3
['High Carb']

Swiss Cheese 
1
['High Carb']

Traditional Hummus 
16
[]

Tuna Salad
4.8
['High Carb']

Turkey Deli Meat
1
['High Carb']

Chipotle Chicken Pasta
31.4
[]

Classic Marinara
10.6
['High Carb']

Garlic Bread
18.9
[]

Linguine Noodles
46.6
[]

Seared Chicken Breast
0
['High Carb']

Three Cheese Alfredo Sauce
9.8
['High Carb']

Ziti
35
[]

Gluten Friendly Cheese 